# skimage and shapely 
- skimage: main purpose is image processing, so the underlying data stored in np.array is interpreted in pixel cooridnate system
- shapely: more "mathematical" treatment on polygons and geometric operations
    - many of the strong geospatial libraries (eg. geopandas, osmnx, rasterio) uses some kind of derivatives of `fiona` or `gdal` etc, on which `shapely` is also built upon. 
    - it seems more natural to use with those geospatial data (ie. data with actual earth coordinates attached)
    - less headache to think about crs conversion between image crs and cartesian coordinate system)or vector (or line and other shapes in cartesian coordinate)

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts
import panel as pn
hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)
    

In [ ]:
from utils import get_mro as mro, nprint
import utils as u
from vector import Vector as vec

In [ ]:
# list_cmaps(provider='colorcet', category='Sequential')

In [ ]:
%opts Image [colorbar=True, tools=['hover']] Curve [tools=['hover']]

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

In [ ]:
class Line2d():
    """ 
    Representation of a line in two dimsional space defined by two end points
    """
    
    def __init__(self, p0, p1):
        """"
        Args:
        - p0 (2dim vec)
        - p1 (2dim vec)
        """
        assert p0.ndim == 2 and p1.ndim == 2
        self.p0 = p0
        self.p1 = p1
        
    def length(self):
        return (p0-p1).norm
    
    def slope(self):
        return (p1[1]-p0[1])/(p1[0]-p0[0])
    
    def unit_tangent(self):
        """Unit tangent(ie. velocity vector)
        """
        return (p1-p0).normalize()
        
    def unit_normal(self):
        x,y = self.unit_tangent()
        return vec(-y,x)
    
    def get_normal_band(self, distance):
        """
        Use the normal vector as the directional vector to construct a bbox around this line
        - distance (float)
        """
        return self.get_band(self.unit_normal(), distance)
    
    def get_band(self, direction, distance):
        """
        Args:
        - direction (2d vec): Doesn't have to be of unit length as we will do the normalization again.
        - distance (positive float)
        """
        assert direction.ndim == 2 and distance >= 0
        direction = direction.normalize() 
        b0 = self.p0 + direction
        b1 = self.p0 - direction
        b2 = self.p1 - direction
        b3 = self.p1 + direction
        
        return (b0, b1, b2, b3)
    
    def hvplot(self, **opts):
        return hv.Curve([self.p0, self.p1]).opts(**opts).opts(padding=0.1, aspect='equal')#,data_aspect=1))
    
    def __repr__(self):
        return f"Line2d({self.p0.values}, {self.p1.values})"
        
    

In [ ]:
p0 = vec(0,0)
p1 = vec(2,1)
l = Line2d(p0,p1)

In [ ]:
l

In [ ]:
l.hvplot() * p0.hvplot(color='b', size=10) * p1.hvplot(color='r', size=10) *p1.rotate(90).hvplot(color='g', size=10)

In [ ]:
l2 = Line2d(p0, p1.rotate(90))
(
    l.hvplot() 
    * p0.hvplot(color='b', size=10) 
    * p1.hvplot(color='r', size=10) 
    *p1.rotate(90).hvplot(color='g', size=10)
    * l2.hvplot()
)

In [ ]:
t = l.unit_tangent()
n = l.unit_normal()
(
    l.hvplot() 
    * t.hvplot(color='r')
    * n.hvplot(color='g')
    
)

In [ ]:
t.norm(), n.norm()

Let's get a bbox of a buffer around the line object.

![line_buffer](../assets/line_buffer.png)

In [ ]:
band = l.get_normal_band(1.)

In [ ]:
def test_Line2d_get_normal_band_1():
    from shapely.geometry import Polygon
    p0 = vec(0,0)
    p1 = vec(0,1)

    l = Line2d(p0,p1)
    band = l.get_normal_band(distance=1)
    overlay = (
        l.hvplot() 
        * l.unit_normal().hvplot(color='r', size=10)
        * hv.Polygons([Polygon(band)]).opts(alpha=0.1)
    )
    display(overlay)
    
def test_Line2d_get_normal_band_2():
    from shapely.geometry import Polygon

    p0 = vec(0,0)
    p1 = vec(2,1)

    l = Line2d(p0,p1)
    band = l.get_normal_band(1)
    overlay = (
        l.hvplot() 
        * l.unit_normal().hvplot(color='r', size=10)
        * hv.Polygons([Polygon(band)]).opts(alpha=0.1)
    )
    display(overlay)
    


In [ ]:
test_Line2d_get_normal_band_2()

In [ ]:
p0 = vec(0,0)
p1 = vec(2,1)

l = Line2d(p0,p1)
band = l.get_normal_band(1)

In [ ]:
(
    l.hvplot() 
    *l.unit_normal().hvplot(color='r', size=10)
    * hv.Polygons([Polygon(band)]).opts(alpha=0.1)
        
)

In [ ]:
Polygon(band)

In [ ]:
hv.Polygons([Polygon(band)])

In [ ]:
band[0]

In [ ]:
b0 = band[0]

In [ ]:
print(b0)

In [ ]:
type(b0)